In [1]:
import pandas as pd


In [2]:
df = pd.read_csv("../../data/covid19/covid19mexico.csv.gz",index_col=0)


C:\Users\Meenakshi\AppData\Local\Temp\ipykernel_24292\1444622623.py:1: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../../data/covid19/covid19mexico.csv.gz",index_col=0)


In [3]:
df.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2023-07-18,0ed066,2,12,16,2,16,16,41,1,...,2,2,97,1,2,7,99,México,97,97
1,2023-07-18,9edc59,2,12,7,1,7,7,108,1,...,2,2,97,2,97,6,99,México,97,97
2,2023-07-18,72b451,1,12,22,1,22,22,6,1,...,2,1,2,1,2,7,99,México,97,97
3,2023-07-18,dd7e13,2,6,16,2,16,16,112,1,...,2,2,97,1,1,3,99,México,97,97
4,2023-07-18,62dc8d,2,12,7,2,7,7,64,1,...,2,2,97,2,97,6,99,México,97,97


In [4]:
df.shape

(985570, 40)

In [5]:
list(df.columns)

['FECHA_ACTUALIZACION',
 'ID_REGISTRO',
 'ORIGEN',
 'SECTOR',
 'ENTIDAD_UM',
 'SEXO',
 'ENTIDAD_NAC',
 'ENTIDAD_RES',
 'MUNICIPIO_RES',
 'TIPO_PACIENTE',
 'FECHA_INGRESO',
 'FECHA_SINTOMAS',
 'FECHA_DEF',
 'INTUBADO',
 'NEUMONIA',
 'EDAD',
 'NACIONALIDAD',
 'EMBARAZO',
 'HABLA_LENGUA_INDIG',
 'INDIGENA',
 'DIABETES',
 'EPOC',
 'ASMA',
 'INMUSUPR',
 'HIPERTENSION',
 'OTRA_COM',
 'CARDIOVASCULAR',
 'OBESIDAD',
 'RENAL_CRONICA',
 'TABAQUISMO',
 'OTRO_CASO',
 'TOMA_MUESTRA_LAB',
 'RESULTADO_LAB',
 'TOMA_MUESTRA_ANTIGENO',
 'RESULTADO_ANTIGENO',
 'CLASIFICACION_FINAL',
 'MIGRANTE',
 'PAIS_NACIONALIDAD',
 'PAIS_ORIGEN',
 'UCI']

In [6]:
# Spanish Columns Name -> English
tr_cols = """FECHA_ACTUALIZACION -> DATE_UPDATES;\
ID_REGISTRO -> REGISTRATION_ID;\
ORIGEN -> ORIGEN;\
SECTOR -> SECTOR;\
ENTIDAD_UM -> ENTITY_ONE;\
SEXO -> SEX;\
ENTIDAD_NAC -> ENTITY_NAC;\
ENTIDAD_RES -> ENTITY_RES;\
MUNICIPIO_RES -> MUNICIPAL_RES;\
TIPO_PACIENTE -> PATIENT_TYPE;\
FECHA_INGRESO -> ADMISSION DATE;\
FECHA_SINTOMAS -> CLOSE_SYMPTOMS;\
FECHA_DEF -> CLOSE_DEF;\
INTUBADO -> INTUBATED;\
NEUMONIA -> PNEUMONIA;\
EDAD -> AGE;\
NACIONALIDAD -> NATIONALITY;\
EMBARAZO -> PREGNANCY;\
HABLA_LENGUA_INDIG -> SPEAK_INDIG_LANGUAGE;\
INDIGENA -> INDIGENOUS;\
DIABETES -> DIABETES;\
EPOC -> COPD;\
ASMA -> ASMA;\
INMUSUPR -> INMUSUPR;\
HIPERTENSION -> HYPERTENSION;\
OTRA_COM -> OTHER_COM;\
CARDIOVASCULAR -> CARDIOVASCULAR;\
OBESIDAD -> OBESITY;\
RENAL_CRONICA -> CRONIC RENALS;\
TABAQUISMO -> SMOKING;\
OTRO_CASO -> ANOTHER CASE;\
TOMA_MUESTRA_LAB -> TAKE_SAMPLE_LAB;\
RESULTADO_LAB -> RESULT_LAB;\
TOMA_MUESTRA_ANTIGENO -> TAKE_SAMPLE_ANTIGEN;\
RESULTADO_ANTIGENO -> ANTIGEN_RESULT;\
CLASIFICACION_FINAL -> FINAL_CLASSIFICATION;\
MIGRANTE -> MIGRANT;\
PAIS_NACIONALIDAD -> COUNTRY_NATIONALITY;\
PAIS_ORIGEN -> PAIS_ORIGEN;\
UCI -> UCI"""

In [7]:
tr_pairs = tr_cols.split(';')

In [8]:
tr_pairs

['FECHA_ACTUALIZACION -> DATE_UPDATES',
 'ID_REGISTRO -> REGISTRATION_ID',
 'ORIGEN -> ORIGEN',
 'SECTOR -> SECTOR',
 'ENTIDAD_UM -> ENTITY_ONE',
 'SEXO -> SEX',
 'ENTIDAD_NAC -> ENTITY_NAC',
 'ENTIDAD_RES -> ENTITY_RES',
 'MUNICIPIO_RES -> MUNICIPAL_RES',
 'TIPO_PACIENTE -> PATIENT_TYPE',
 'FECHA_INGRESO -> ADMISSION DATE',
 'FECHA_SINTOMAS -> CLOSE_SYMPTOMS',
 'FECHA_DEF -> CLOSE_DEF',
 'INTUBADO -> INTUBATED',
 'NEUMONIA -> PNEUMONIA',
 'EDAD -> AGE',
 'NACIONALIDAD -> NATIONALITY',
 'EMBARAZO -> PREGNANCY',
 'HABLA_LENGUA_INDIG -> SPEAK_INDIG_LANGUAGE',
 'INDIGENA -> INDIGENOUS',
 'DIABETES -> DIABETES',
 'EPOC -> COPD',
 'ASMA -> ASMA',
 'INMUSUPR -> INMUSUPR',
 'HIPERTENSION -> HYPERTENSION',
 'OTRA_COM -> OTHER_COM',
 'CARDIOVASCULAR -> CARDIOVASCULAR',
 'OBESIDAD -> OBESITY',
 'RENAL_CRONICA -> CRONIC RENALS',
 'TABAQUISMO -> SMOKING',
 'OTRO_CASO -> ANOTHER CASE',
 'TOMA_MUESTRA_LAB -> TAKE_SAMPLE_LAB',
 'RESULTADO_LAB -> RESULT_LAB',
 'TOMA_MUESTRA_ANTIGENO -> TAKE_SAMPLE_ANT

In [9]:
eng_cols,sp_cols = [],[]
for cols in tr_pairs:
    eng_name = cols.split(' ')
    eng = eng_name[-1]
    sp = eng_name[0]
    sp_cols.append(sp)
    eng_cols.append(eng)

In [10]:
eng_cols

['DATE_UPDATES',
 'REGISTRATION_ID',
 'ORIGEN',
 'SECTOR',
 'ENTITY_ONE',
 'SEX',
 'ENTITY_NAC',
 'ENTITY_RES',
 'MUNICIPAL_RES',
 'PATIENT_TYPE',
 'DATE',
 'CLOSE_SYMPTOMS',
 'CLOSE_DEF',
 'INTUBATED',
 'PNEUMONIA',
 'AGE',
 'NATIONALITY',
 'PREGNANCY',
 'SPEAK_INDIG_LANGUAGE',
 'INDIGENOUS',
 'DIABETES',
 'COPD',
 'ASMA',
 'INMUSUPR',
 'HYPERTENSION',
 'OTHER_COM',
 'CARDIOVASCULAR',
 'OBESITY',
 'RENALS',
 'SMOKING',
 'CASE',
 'TAKE_SAMPLE_LAB',
 'RESULT_LAB',
 'TAKE_SAMPLE_ANTIGEN',
 'ANTIGEN_RESULT',
 'FINAL_CLASSIFICATION',
 'MIGRANT',
 'COUNTRY_NATIONALITY',
 'PAIS_ORIGEN',
 'UCI']

In [11]:
#Change column names
df.columns = eng_cols

In [12]:
df.head()

,DATE_UPDATES,REGISTRATION_ID,ORIGEN,SECTOR,ENTITY_ONE,SEX,ENTITY_NAC,ENTITY_RES,MUNICIPAL_RES,PATIENT_TYPE,...,CASE,TAKE_SAMPLE_LAB,RESULT_LAB,TAKE_SAMPLE_ANTIGEN,ANTIGEN_RESULT,FINAL_CLASSIFICATION,MIGRANT,COUNTRY_NATIONALITY,PAIS_ORIGEN,UCI
0,2023-07-18,0ed066,2,12,16,2,16,16,41,1,...,2,2,97,1,2,7,99,México,97,97
1,2023-07-18,9edc59,2,12,7,1,7,7,108,1,...,2,2,97,2,97,6,99,México,97,97
2,2023-07-18,72b451,1,12,22,1,22,22,6,1,...,2,1,2,1,2,7,99,México,97,97
3,2023-07-18,dd7e13,2,6,16,2,16,16,112,1,...,2,2,97,1,1,3,99,México,97,97
4,2023-07-18,62dc8d,2,12,7,2,7,7,64,1,...,2,2,97,2,97,6,99,México,97,97


In [13]:
# Drop some unnecessary columns
# Dropping column names:
# DATE_UPDATES, REGISTRATION_ID,ENTITY_NAC,ENTITY_RES,MUNICIPAL_RES,
# ORIGEN,SECTOR,ENTITY_ONE,DATE,CLOSE_SYMPTOMS,CLOSE_DEF,NATIONALITY,
# SPEAK_INDIG_LANGUAGE,INDIGENOUS,CASE,TAKE_SAMPLE_LAB,RESULT_LAB,
# TAKE_SAMPLE_ANTIGEN,ANTIGEN_RESULT,MIGRANT,COUNTRY_NATIONALITY,PAIS_ORIGEN,PATIENT_TYPE

In [14]:
new_df = df.drop(['DATE_UPDATES', 'REGISTRATION_ID','ENTITY_NAC','ENTITY_RES','MUNICIPAL_RES',
'ORIGEN','SECTOR','ENTITY_ONE','DATE','CLOSE_SYMPTOMS','CLOSE_DEF','NATIONALITY',
'SPEAK_INDIG_LANGUAGE','INDIGENOUS','CASE','TAKE_SAMPLE_LAB','RESULT_LAB','INTUBATED',
'TAKE_SAMPLE_ANTIGEN','ANTIGEN_RESULT','MIGRANT','COUNTRY_NATIONALITY','PAIS_ORIGEN','PATIENT_TYPE','UCI'],axis=1)

In [15]:
new_df.head(2)

,SEX,PNEUMONIA,AGE,PREGNANCY,DIABETES,COPD,ASMA,INMUSUPR,HYPERTENSION,OTHER_COM,CARDIOVASCULAR,OBESITY,RENALS,SMOKING,FINAL_CLASSIFICATION
0,2,2,28,97,2,2,2,2,2,2,2,2,2,1,7
1,1,2,66,2,2,2,2,2,2,2,2,2,2,2,6


In [16]:
# Change the Data types to INT
new_df[['PNEUMONIA']] = df[['PNEUMONIA']].apply(pd.to_numeric)
new_df[['PREGNANCY']] = df[['PREGNANCY']].apply(pd.to_numeric) 
new_df[['DIABETES']] = df[['DIABETES']].apply(pd.to_numeric)
new_df[['COPD']] = df[['COPD']].apply(pd.to_numeric)
new_df[['ASMA']] = df[['ASMA']].apply(pd.to_numeric)
new_df[['INMUSUPR']] = df[['INMUSUPR']].apply(pd.to_numeric) 
new_df[['HYPERTENSION']] = df[['HYPERTENSION']].apply(pd.to_numeric)
new_df[['OTHER_COM']] = df[['OTHER_COM']].apply(pd.to_numeric) 

new_df[['CARDIOVASCULAR']] = df[['CARDIOVASCULAR']].apply(pd.to_numeric)
new_df[['OBESITY']] = df[['OBESITY']].apply(pd.to_numeric) 
new_df[['RENALS']] = df[['RENALS']].apply(pd.to_numeric)
new_df[['SMOKING']] = df[['SMOKING']].apply(pd.to_numeric)

In [17]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 985570 entries, 0 to 985569
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype
---  ------                --------------   -----
 0   SEX                   985570 non-null  int64
 1   PNEUMONIA             985570 non-null  int64
 2   AGE                   985570 non-null  int64
 3   PREGNANCY             985570 non-null  int64
 4   DIABETES              985570 non-null  int64
 5   COPD                  985570 non-null  int64
 6   ASMA                  985570 non-null  int64
 7   INMUSUPR              985570 non-null  int64
 8   HYPERTENSION          985570 non-null  int64
 9   OTHER_COM             985570 non-null  int64
 10  CARDIOVASCULAR        985570 non-null  int64
 11  OBESITY               985570 non-null  int64
 12  RENALS                985570 non-null  int64
 13  SMOKING               985570 non-null  int64
 14  FINAL_CLASSIFICATION  985570 non-null  int64
dtypes: int64(15)
memory usage: 120.3 M

In [18]:
# Convert SEX to proper numeric: Here 2 -> Male, 1 -> Female. We change 0 -> Female and 1 -> Male
# No missing entries for SEX
fem = list((new_df.loc[new_df['SEX'] == 1]).index)
new_df.loc[fem,["SEX"]] = 0 
male = list((new_df.loc[new_df['SEX'] == 2]).index)
new_df.loc[male,["SEX"]] = 1 

In [19]:
# Convert PNEUMONIA	to proper numeric: Here 2 -> NO, 1 -> YES, 99 -> "Missing". 
# We change 2 -> 0(NO), None -> "Missing"
# 5148 Missing Records For Pneumonia
nopne = list((new_df.loc[new_df['PNEUMONIA'] == 2]).index)
new_df.loc[nopne,["PNEUMONIA"]] = 0 
miss = list((new_df.loc[new_df['PNEUMONIA'] == 99]).index)
new_df.loc[miss,["PNEUMONIA"]] = None 

In [20]:
# Convert PREGNANCY	to proper numeric: Here 2 -> NO, 1 -> YES. 97,98,99 -> "Missing". 
# We change 0 -> No and None -> "Missing"
nopre = list((new_df.loc[new_df['PREGNANCY'] == 2]).index)
new_df.loc[nopre,["PREGNANCY"]] = 0 
miss = list((new_df.loc[(new_df['PREGNANCY'] == 99) | (new_df['PREGNANCY'] == 98) | (new_df['PREGNANCY'] == 97)]).index)
new_df.loc[miss,["PREGNANCY"]] = None 

In [21]:
# Convert DIABETES	to proper numeric: Here 2 -> NO, 1 -> YES. 98 -> "Missing". 
# We change 0 -> No and None -> "Missing"
nodie = list((new_df.loc[new_df['DIABETES'] == 2]).index)
new_df.loc[nodie,["DIABETES"]] = 0 
miss = list((new_df.loc[(new_df['DIABETES'] == 98)]).index)
new_df.loc[miss,["DIABETES"]] = None 

In [22]:
# Convert COPD to proper numeric: Here 2 -> NO, 1 -> YES. 98 -> "Missing". 
# We change 0 -> No and None -> "Missing"
nocopd = list((new_df.loc[new_df['COPD'] == 2]).index)
new_df.loc[nocopd,["COPD"]] = 0 
miss = list((new_df.loc[(new_df['COPD'] == 98)]).index)
new_df.loc[miss,["COPD"]] = None 

In [23]:
# Convert ASMA to proper numeric: Here 2 -> NO, 1 -> YES. 98 -> "Missing". 
# We change 0 -> No and None -> "Missing"
noasma = list((new_df.loc[new_df['ASMA'] == 2]).index)
new_df.loc[noasma,["ASMA"]] = 0 
miss = list((new_df.loc[(new_df['ASMA'] == 98)]).index)
new_df.loc[miss,["ASMA"]] = None 

In [24]:
# Convert INMUSUPR to proper numeric: Here 2 -> NO, 1 -> YES. 98 -> "Missing". 
# We change 0 -> No and None -> "Missing"
noimm = list((new_df.loc[new_df['INMUSUPR'] == 2]).index)
new_df.loc[noimm,["INMUSUPR"]] = 0 
miss = list((new_df.loc[(new_df['INMUSUPR'] == 98)]).index)
new_df.loc[miss,["INMUSUPR"]] = None 

In [25]:
# Convert HYPERTENSION to proper numeric: Here 2 -> NO, 1 -> YES. 98 -> "Missing". 
# We change 0 -> No and None -> "Missing"
nohy = list((new_df.loc[new_df['HYPERTENSION'] == 2]).index)
new_df.loc[nohy,["HYPERTENSION"]] = 0 
miss = list((new_df.loc[(new_df['HYPERTENSION'] == 98)]).index)
new_df.loc[miss,["HYPERTENSION"]] = None 

In [26]:
# Convert OTHER_COM to proper numeric: Here 2 -> NO, 1 -> YES. 98 -> "Missing". 
# We change 0 -> No and None -> "Missing"
no = list((new_df.loc[new_df['OTHER_COM'] == 2]).index)
new_df.loc[no,["OTHER_COM"]] = 0 
miss = list((new_df.loc[(new_df['OTHER_COM'] == 98)]).index)
new_df.loc[miss,["OTHER_COM"]] = None 

In [27]:
# Convert CARDIOVASCULAR to proper numeric: Here 2 -> NO, 1 -> YES. 98 -> "Missing". 
# We change 0 -> No and None -> "Missing"
nocar = list((new_df.loc[new_df['CARDIOVASCULAR'] == 2]).index)
new_df.loc[nocar,["CARDIOVASCULAR"]] = 0 
miss = list((new_df.loc[(new_df['CARDIOVASCULAR'] == 98)]).index)
new_df.loc[miss,["CARDIOVASCULAR"]] = None 

In [28]:
# Convert OBESITY to proper numeric: Here 2 -> NO, 1 -> YES. 98 -> "Missing". 
# We change 0 -> No and None -> "Missing"
noobe = list((new_df.loc[new_df['OBESITY'] == 2]).index)
new_df.loc[noobe,["OBESITY"]] = 0 
miss = list((new_df.loc[(new_df['OBESITY'] == 98)]).index)
new_df.loc[miss,["OBESITY"]] = None 

In [29]:
# Convert RENALS to proper numeric: Here 2 -> NO, 1 -> YES. 98 -> "Missing". 
# We change 0 -> No and None -> "Missing"
norene = list((new_df.loc[new_df['RENALS'] == 2]).index)
new_df.loc[norene,["RENALS"]] = 0 
miss = list((new_df.loc[(new_df['RENALS'] == 98)]).index)
new_df.loc[miss,["RENALS"]] = None 

In [30]:
# Convert SMOKING to proper numeric: Here 2 -> NO, 1 -> YES. 98 -> "Missing". 
# We change 0 -> No and None -> "Missing"
nosmo = list((new_df.loc[new_df['SMOKING'] == 2]).index)
new_df.loc[nosmo,["SMOKING"]] = 0 
miss = list((new_df.loc[(new_df['SMOKING'] == 98)]).index)
new_df.loc[miss,["SMOKING"]] = None 

In [31]:
# Convert FINAL_CLASSIFICATION to proper numeric: Here Values 1-3 mean that the patient was
# diagnosed with covid in different degrees. 
# 4 or higher means that the patient is not a carrier of covid or that the test is inconclusive.
# We change 1-3 -> 1 rest all 0. 
yescov = list((new_df.loc[(new_df['FINAL_CLASSIFICATION']<4)]).index)
new_df.loc[yescov,["FINAL_CLASSIFICATION"]] = 1 
nocov = list((new_df.loc[new_df['FINAL_CLASSIFICATION']>=4]).index)
new_df.loc[nocov,["FINAL_CLASSIFICATION"]] = 0 

In [32]:
new_df["FINAL_CLASSIFICATION"].value_counts()

0    627501
1    358069
Name: FINAL_CLASSIFICATION, dtype: int64

In [33]:
new_df.shape

(985570, 15)

In [34]:
new_df.head(10)

,SEX,PNEUMONIA,AGE,PREGNANCY,DIABETES,COPD,ASMA,INMUSUPR,HYPERTENSION,OTHER_COM,CARDIOVASCULAR,OBESITY,RENALS,SMOKING,FINAL_CLASSIFICATION
0,1,0.0,28,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
1,0,0.0,66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0,0.0,27,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,1,0.0,47,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,1,0.0,53,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
5,0,0.0,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
6,0,0.0,72,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
7,0,0.0,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
8,1,0.0,33,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
9,0,0.0,22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [35]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 985570 entries, 0 to 985569
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   SEX                   985570 non-null  int64  
 1   PNEUMONIA             980422 non-null  float64
 2   AGE                   985570 non-null  int64  
 3   PREGNANCY             572920 non-null  float64
 4   DIABETES              984625 non-null  float64
 5   COPD                  984709 non-null  float64
 6   ASMA                  984750 non-null  float64
 7   INMUSUPR              984777 non-null  float64
 8   HYPERTENSION          984727 non-null  float64
 9   OTHER_COM             978704 non-null  float64
 10  CARDIOVASCULAR        984743 non-null  float64
 11  OBESITY               984810 non-null  float64
 12  RENALS                984742 non-null  float64
 13  SMOKING               984695 non-null  float64
 14  FINAL_CLASSIFICATION  985570 non-null  int64  
dtype

In [36]:
new_cols = list(new_df.columns)

In [37]:
#df.to_csv('covid19mexico.csv.gz', compression='gzip',index=False)
new_df.FINAL_CLASSIFICATION.value_counts()

0    627501
1    358069
Name: FINAL_CLASSIFICATION, dtype: int64

In [38]:
TRAIN_FRACTION = 0.5
RANDOM_SEED = 56732
train_df = new_df.sample(frac=TRAIN_FRACTION,random_state=RANDOM_SEED)

In [51]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imp = IterativeImputer(max_iter=10, random_state=RANDOM_SEED)
 
imp.fit(train_df[new_cols[:-1]])
train_x_df = imp.transform(train_df[new_cols[:-1]])

In [50]:
display(train_df.info())
display(pd.DataFrame(train_x_df,columns=new_cols[:-1]).info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 492785 entries, 75083 to 169968
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   SEX                   492785 non-null  int64  
 1   PNEUMONIA             490216 non-null  float64
 2   AGE                   492785 non-null  int64  
 3   PREGNANCY             286632 non-null  float64
 4   DIABETES              492314 non-null  float64
 5   COPD                  492363 non-null  float64
 6   ASMA                  492379 non-null  float64
 7   INMUSUPR              492391 non-null  float64
 8   HYPERTENSION          492355 non-null  float64
 9   OTHER_COM             489360 non-null  float64
 10  CARDIOVASCULAR        492383 non-null  float64
 11  OBESITY               492407 non-null  float64
 12  RENALS                492377 non-null  float64
 13  SMOKING               492344 non-null  float64
 14  FINAL_CLASSIFICATION  492785 non-null  int64  
d

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 492785 entries, 0 to 492784
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   SEX             492785 non-null  float64
 1   PNEUMONIA       492785 non-null  float64
 2   AGE             492785 non-null  float64
 3   PREGNANCY       492785 non-null  float64
 4   DIABETES        492785 non-null  float64
 5   COPD            492785 non-null  float64
 6   ASMA            492785 non-null  float64
 7   INMUSUPR        492785 non-null  float64
 8   HYPERTENSION    492785 non-null  float64
 9   OTHER_COM       492785 non-null  float64
 10  CARDIOVASCULAR  492785 non-null  float64
 11  OBESITY         492785 non-null  float64
 12  RENALS          492785 non-null  float64
 13  SMOKING         492785 non-null  float64
dtypes: float64(14)
memory usage: 52.6 MB


None

In [39]:
train_df.FINAL_CLASSIFICATION.value_counts()

0    313589
1    179196
Name: FINAL_CLASSIFICATION, dtype: int64

In [40]:
train_df.index

Int64Index([ 75083, 478410, 167333, 183458, 245355, 929762, 418462, 750213,
            859368, 979867,
            ...
            783863, 130126, 469556, 944086, 296070, 511542, 742465, 494034,
             77894, 169968],
           dtype='int64', length=492785)

In [41]:
test_df = new_df.drop(index=train_df.index)

In [43]:
import numpy as np
test_df=test_df.sample(frac=1,random_state=RANDOM_SEED)
test_collection=np.array_split(test_df,50)

In [45]:
for test in test_collection:
    num_covid,num_non_covid=test.FINAL_CLASSIFICATION.value_counts()
    print(num_covid/num_non_covid)

1.7099257629914766
1.7002739726027398
1.7654320987654322
1.769317223939309
1.7294378288562724
1.777120315581854
1.7677618646447628
1.797615668464377
1.751535455053043
1.7234042553191489
1.721148536720044
1.7724331926863572
1.784180790960452
1.7873303167420815
1.777903043968433
1.7446393762183237
1.7362576346474181
1.7377777777777779
1.757694459988808
1.8945668135095448
1.7024952015355086
1.7166482910694598
1.784180790960452
1.7662082514734774
1.751535455053043
1.7271721084670726
1.7920679886685553
1.733222407099279
1.718146718146718
1.7256637168141593
1.7249101465302736
1.7952353942144073
1.6659453611035975
1.7301939058171745
1.8119828815977175
1.7705931965139161
1.742833286946841
1.7299168975069252
1.8060933940774488
1.8376043766196373
1.7902038505096263
1.7405450500556174
1.81169757489301
1.7044456641053787
1.859837492745212
1.7344617092119867
1.686017988552739
1.7902038505096263
1.7
1.7261410788381744


In [ ]:
[depth, train_accuracy,test1_accuracy,test2_accuracy,....,test50_accuracy,train_recall,test1_recall,test2_recall.....,test50_recall,